<a href="https://colab.research.google.com/github/yeialgov/MAYC_ML_Revenue_Prediction/blob/EDA/Yeisson_Copy_of_MAYC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing kaggle data 

In [2]:
#Mount the Google Drive to Google Colab
#1. get the library
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_path = '/content/gdrive/My Drive/ReDi_DA_Project'

In [4]:
cd '/content/gdrive/My Drive/ReDi_DA_Project'

/content/gdrive/My Drive/ReDi_DA_Project


In [0]:
#for checking the Data
ls -alFh

total 98M
drwx------ 2 root root 4.0K May  8 14:47 __MACOSX/
drwx------ 2 root root 4.0K May  8 14:47 preprocessed_ga_revenue_data/
-r-------- 1 root root  98M May  4 09:14 preprocessed_ga_revenue_data.zip


In [0]:
#! unzip train.csv.zip
! unzip preprocessed_ga_revenue_data.zip

Archive:  preprocessed_ga_revenue_data.zip
replace preprocessed_ga_revenue_data/train_flat_no_hits.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/preprocessed_ga_revenue_data/._train_flat_no_hits.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
ls

__MACOSX/  preprocessed_ga_revenue_data/  preprocessed_ga_revenue_data.zip


In [5]:
cd preprocessed_ga_revenue_data

/content/gdrive/My Drive/ReDi_DA_Project/preprocessed_ga_revenue_data


In [6]:
ls

test_flat_no_hits.pkl  train_flat_no_hits.pkl
test_hits_flat.pkl     train_hits_flat.pkl


# Creating dfs

In [0]:
# download the libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#!pip3 install --upgrade seaborn cufflinks plotly
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
#?
import requests

In [0]:
# load training data set info a df
#df_train=pd.read_csv(filepath_or_buffer='/content/gdrive/My Drive/DA-Data/Kaggel_test.csv')
df_train = pd.read_pickle('train_flat_no_hits.pkl')
#df_test = pd.read_pickle('test_flat_no_hits.pkl')

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 60 columns):
 #   Column                                             Dtype 
---  ------                                             ----- 
 0   channelGrouping                                    object
 1   date                                               int64 
 2   fullVisitorId                                      object
 3   socialEngagementType                               object
 4   visitId                                            int64 
 5   visitNumber                                        int64 
 6   visitStartTime                                     int64 
 7   device.browser                                     object
 8   device.browserVersion                              object
 9   device.browserSize                                 object
 10  device.operatingSystem                             object
 11  device.operatingSystemVersion                      object
 12  

In [0]:
df_train.head(5)

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.campaignCode,customDimension.index,customDimension.value
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,1,NaN,NaN,NaN,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,EMEA
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,NaN,NaN,2,28,NaN,NaN,NaN,(not set),sites.google.com,referral,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,North America
2,Direct,20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,NaN,1,1,38,NaN,NaN,NaN,(not set),(direct),(none),NaN,not available in demo dataset,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Nort

In [0]:
df_train['totals.transactionRevenue'].value_counts()

16990000     308
19990000     248
39980000     220
18990000     219
21990000     211
            ... 
181940000      1
131710000      1
449700000      1
494620000      1
247900000      1
Name: totals.transactionRevenue, Length: 7251, dtype: int64

In [0]:
df_train.channelGrouping.unique()

array(['Organic Search', 'Referral', 'Direct', 'Paid Search', 'Display',
       'Affiliates', 'Social', '(Other)'], dtype=object)

**fullVisitorId**- A unique identifier for each user of the Google Merchandise Store.
**channelGrouping **- The channel via which the user came to the Store.


**geoNetwork** - This section contains information about the geography of the user.
**socialEngagementType** - Engagement type, either "Socially Engaged" or "Not Socially Engaged".


visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. *For a completely unique ID, you should use a combination of fullVisitorId and visitId*.
visitNumber - The session number for this user. If this is the first session, then this is set to 1.
visitStartTime - The timestamp (expressed as POSIX time).


totals - This set of columns mostly includes high-level aggregate data.

In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 60 columns):
 #   Column                                             Dtype 
---  ------                                             ----- 
 0   channelGrouping                                    object
 1   date                                               int64 
 2   fullVisitorId                                      object
 3   socialEngagementType                               object
 4   visitId                                            int64 
 5   visitNumber                                        int64 
 6   visitStartTime                                     int64 
 7   device.browser                                     object
 8   device.browserVersion                              object
 9   device.browserSize                                 object
 10  device.operatingSystem                             object
 11  device.operatingSystemVersion                      object
 12  

# Yeisson EDA of geograpfical info

In [0]:
Geo_columns=['geoNetwork.continent',	'geoNetwork.subContinent',	'geoNetwork.country',	'geoNetwork.region',	'geoNetwork.metro',	'geoNetwork.city',	'geoNetwork.cityId'	,'geoNetwork.networkDomain',	'geoNetwork.latitude',	'geoNetwork.longitude','geoNetwork.networkLocation']

In [0]:
df_train[Geo_columns].agg(['count', 'size', 'nunique'])

,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation
count,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337
size,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337
nunique,6,23,228,483,123,956,1,41982,1,1,1


In [0]:
#update the Geo_column list without the not available values
Geo_columns=['geoNetwork.continent',	'geoNetwork.subContinent',	'geoNetwork.country',	'geoNetwork.region',	'geoNetwork.metro',	'geoNetwork.city',	'geoNetwork.networkDomain']

In [11]:
df_train[Geo_columns].agg(['count', 'size', 'nunique'])

,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain
count,1708337,1708337,1708337,1708337,1708337,1708337,1708337
size,1708337,1708337,1708337,1708337,1708337,1708337,1708337
nunique,6,23,228,483,123,956,41982


In [0]:
#transform the objective variable into a numerical variable
df_train["totals.transactionRevenue"] = df_train["totals.transactionRevenue"].astype('float')
gdf=df_train.groupby(['geoNetwork.continent'])[['totals.transactionRevenue',]].sum()

In [0]:
A=df_train.groupby(['geoNetwork.networkDomain'])[['totals.transactionRevenue']].count()
A=A[A['totals.transactionRevenue']>0]
A.sort_values(by='totals.transactionRevenue', ascending=False, inplace=True)
A

,totals.transactionRevenue
geoNetwork.networkDomain,
(not set),10867
comcast.net,1505
verizon.net,755
unknown.unknown,665
rr.com,588
...,...
mtco.com,1
mtnsat.com,1
mtroyal.ca,1


In [0]:
B=df_train.groupby(['geoNetwork.networkDomain'])[['totals.transactionRevenue']].sum()
B=B[B['totals.transactionRevenue']>0]
B.sort_values(by='totals.transactionRevenue', ascending=False, inplace=True)
B

,totals.transactionRevenue
geoNetwork.networkDomain,
(not set),1.305843e+12
comcastbusiness.net,2.079817e+11
comcast.net,1.773147e+11
verizon.net,7.967340e+10
unknown.unknown,5.825398e+10
...,...
netfirms.com,1.990000e+06
hsn.net,1.990000e+06
dvois.com,1.500000e+06


In [0]:
A.merge(B, on='geoNetwork.networkDomain',)


,totals.transactionRevenue_x,totals.transactionRevenue_y
geoNetwork.networkDomain,,
(not set),10867,1.305843e+12
comcast.net,1505,1.773147e+11
verizon.net,755,7.967340e+10
unknown.unknown,665,5.825398e+10
rr.com,588,5.322676e+10
...,...,...
mtco.com,1,1.099000e+07
mtnsat.com,1,1.499000e+07
mtroyal.ca,1,2.658000e+07


## Domain Insight:
- From the 41982 unique domains register only 996 has had at least one transaction.
- the not set domain aggregate a big number of finished transactions

In [0]:
df_train.groupby(['geoNetwork.continent'])[['totals.transactionRevenue',]].count()

,totals.transactionRevenue
geoNetwork.continent,
(not set),7
Africa,12
Americas,18150
Asia,190
Europe,126
Oceania,29


In [0]:
ax=df_train.groupby(['geoNetwork.continent'])[['totals.transactionRevenue',]].sum()

In [0]:
#log1p Calculates natural logarithm (1 + x).

ax['log_n']=ax['totals.transactionRevenue'].apply(lambda x: np.log1p(x))
ax


,totals.transactionRevenue,log_n,log_n_a
geoNetwork.continent,,,
(not set),7.952200e+08,20.494129,20.494129
Africa,9.025990e+09,22.923374,22.923374
Americas,2.265098e+12,28.448639,28.448639
Asia,2.339772e+10,23.875904,23.875904
Europe,1.124745e+10,23.143407,23.143407
Oceania,6.781100e+09,22.637405,22.637405


# Yeisson Data Cleaning

## List the Columns to Drop

In [21]:
df_train.agg(['count', 'size', 'nunique'])

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.campaignCode,customDimension.index,customDimension.value
count,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708098,871578,1307430,873063,834043,18559,18514,18514,1708337,1708337,1708337,655557,1708337,566264,534518,64737,75274,75274,75423,75274,75274,1,1375102,1375102
size,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337
nunique,8,638,1323730,1,1665802,457,1667423,129,1,1,24,1,2,1,1,1,1,1,1,1,1,1,3,6,23,228,483,123,956,1,41982,1,1,1,1,297,230,1,1,100,4774,13,7251,8506,33,345,7,4546,1,3196,1,76,11,3,59008,3,1,1,1,5


Create a list with all the columns that are not available in the demo set.

In [56]:

pre_candidate=pd.DataFrame()
pre_candidate['non_unq']= df_train.nunique()
candidate_drop=[]
candidate_drop= pre_candidate[pre_candidate.non_unq==1].index
candidate_drop

Index(['socialEngagementType', 'device.browserVersion', 'device.browserSize',
       'device.operatingSystemVersion', 'device.mobileDeviceBranding',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.flashVersion', 'device.language', 'device.screenColors',
       'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits',
       'totals.bounces', 'totals.newVisits',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.isTrueDirect',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.campaignCode', 'customDimension.index'],
      dtype='object')

In [57]:
df_train[candidate_drop].describe()

,socialEngagementType,device.browserVersion,device.browserSize,device.operatingSystemVersion,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,geoNetwork.cityId,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.bounces,totals.newVisits,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.campaignCode,customDimension.index
count,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,871578,1307430,1708337,534518,75274,1,1375102
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,Not Socially Engaged,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,not available in demo dataset,True,False,11251kjhkvahf,4
freq,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,871578,1307430,1708337,534518,75274,1,1375102


In [78]:
drop_column=[]
for i in candidate_drop:
  if df_train[i].value_counts().item()==len(df_train[i]): #evaluate if All the values in the column are the same, in that case no use for prediction
    drop_column.append(i)
drop_column

['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.operatingSystemVersion',
 'device.mobileDeviceBranding',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.flashVersion',
 'device.language',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [79]:
df_train[drop_column].describe()

,socialEngagementType,device.browserVersion,device.browserSize,device.operatingSystemVersion,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,geoNetwork.cityId,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,trafficSource.adwordsClickInfo.criteriaParameters
count,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,Not Socially Engaged,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,not available in demo dataset
freq,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1708337


The ```drop_column``` list include all the columns that can be dropped from the assessment because does not bring any value into the prediction. 


In [84]:
reduced_df_train = df_train.drop(drop_column, axis=1)
reduced_df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 41 columns):
 #   Column                                        Dtype 
---  ------                                        ----- 
 0   channelGrouping                               object
 1   date                                          int64 
 2   fullVisitorId                                 object
 3   visitId                                       int64 
 4   visitNumber                                   int64 
 5   visitStartTime                                int64 
 6   device.browser                                object
 7   device.operatingSystem                        object
 8   device.isMobile                               bool  
 9   device.deviceCategory                         object
 10  geoNetwork.continent                          object
 11  geoNetwork.subContinent                       object
 12  geoNetwork.country                            object
 13  geoNetwork.r